In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [2]:
from dotenv import load_dotenv
import nest_asyncio
import os

os.environ["SWE_AGENT_CONFIG_DIR"] = "."
os.environ["SWE_AGENT_TOOLS_DIR"] = "tools"
os.environ["SWE_AGENT_TRAJECTORY_DIR"] = "trajectories"

os.makedirs("replays", exist_ok=True)
os.makedirs("trajectories", exist_ok=True)

load_dotenv()
nest_asyncio.apply()

In [ ]:
import art
from art.multidevice import MultiDeviceBackend

from instances import as_instances_iter, get_filtered_swe_smith_instances_df
from rollout import ModelConfig, rollout

instances = (get_filtered_swe_smith_instances_df().sample(fraction=1.0, shuffle=True, seed=42).head(1).pipe(as_instances_iter))

backend = MultiDeviceBackend()
model = art.TrainableModel(
    name="001",
    project="swebench",
    base_model="Qwen/Qwen2.5-14B-Instruct",
    config=ModelConfig()
)
await model.register(backend)

trajectory_groups = await art.gather_trajectory_groups(
    (
        art.TrajectoryGroup(rollout(model, instance) for _ in range(1)) for instance in instances
    ),
    pbar_total_completion_tokens=False
)

INFO 05-29 00:20:15 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-29 00:20:15 [__init__.py:239] Automatically detected platform cuda.
